In [1]:
# --- Setup: GitHub token manually ---
import os
os.environ["GITHUB_TOKEN"] = "ghp_#############"


# --- Dependencies ---
import pandas as pd
import requests
from io import BytesIO
from openpyxl import load_workbook
import sys

# --- Mount GitHub utility module ---
!git clone https://github.com/blainehodder/WCSB_Supply_Demand.git
sys.path.append("/content/WCSB_Supply_Demand")
from utils.github_commit import push_df_to_github

# --- Config ---
base_url = "https://raw.githubusercontent.com/blainehodder/WCSB_Supply_Demand/main/raw_data/st3/ST3_{year}.xlsx"
years = list(range(2017, 2025))

# --- Indent map from your verified structure (row-based) ---
indent_map = {
    6: ("SUPPLY", None, None, "stock"),
    7: ("Opening Inventory", None, None, "stock"),
    9: ("Production", None, None, "flow"),
    10: ("Crude Oil Production", None, None, "flow"),
    11: ("Crude Oil Light", "Production", "Crude Oil Production", "flow"),
    12: ("Crude Oil Medium", "Production", "Crude Oil Production", "flow"),
    13: ("Crude Oil Heavy", "Production", "Crude Oil Production", "flow"),
    14: ("Crude Oil Ultra-Heavy", "Production", "Crude Oil Production", "flow"),
    15: ("Total Crude Oil Production", "Production", "Crude Oil Production", "flow"),
    17: ("Condensate Production", "Production", None, "flow"),
    19: ("Oil Sands Production", "Production", None, "flow"),
    20: ("Nonupgraded", "Production", "Oil Sands Production", "flow"),
    21: ("In Situ Production", "Production", "Nonupgraded", "flow"),
    22: ("Mined Production", "Production", "Nonupgraded", "flow"),
    23: ("Sent for Further Processing", "Production", "Nonupgraded", "flow"),
    24: ("Nonupgraded Total", "Production", "Oil Sands Production", "flow"),
    25: ("Upgraded Production", "Production", "Oil Sands Production", "flow"),
    26: ("Total Oil Sands Production", "Production", "Oil Sands Production", "flow"),
    28: ("Total Production", "Production", None, "flow"),
    30: ("Receipts", None, None, "flow"),
    31: ("Pentanes Plus - Plant/Gathering Process", "Receipts", "Pentanes Plus", "flow"),
    32: ("Pentanes Plus - Fractionation Yield", "Receipts", "Pentanes Plus", "flow"),
    34: ("Skim Oil Recovered", "Receipts", None, "flow"),
    35: ("Waste Plant Receipts", "Receipts", None, "flow"),
    36: ("Other Alberta Receipts", "Receipts", None, "flow"),
    38: ("Butanes reported as Crude Oil or Equivalent", "Receipts", None, "flow"),
    39: ("NGL reported as Crude Oil or Equivalent", "Receipts", None, "flow"),
    41: ("Imports", "Receipts", None, "flow"),
    42: ("Pentanes Plus", "Imports", None, "flow"),
    43: ("Condensates", "Imports", None, "flow"),
    44: ("Crude Oil", "Imports", None, "flow"),
    45: ("Synthetic Crude Oil", "Imports", None, "flow"),
    46: ("Total Imports", "Receipts", None, "flow"),
    48: ("Total Receipts", "Receipts", None, "flow"),
    50: ("Flare or Waste", None, None, "flow"),
    51: ("Fuel", None, None, "flow"),
    52: ("Shrinkage", None, None, "flow"),
    54: ("Closing Inventory", None, None, "stock"),
    55: ("Adjustments", None, None, "flow"),
    57: ("TOTAL OIL & EQUIVALENT SUPPLY", None, None, "flow"),
    61: ("DISPOSITION", None, None, "flow"),
    62: ("Alberta Use", "DISPOSITION", None, "flow"),
    63: ("Alberta Injection and Well Use", "Alberta Use", None, "flow"),
    64: ("Alberta Refinery Sales", "Alberta Use", None, "flow"),
    65: ("Waste Plant Use", "Alberta Use", None, "flow"),
    66: ("Plant Use", "Alberta Use", None, "flow"),
    67: ("Line Fill", "Alberta Use", None, "flow"),
    68: ("Load Fluid", "Alberta Use", None, "flow"),
    69: ("Alberta Other Sales", "Alberta Use", None, "flow"),
    70: ("Total Alberta Use", "DISPOSITION", None, "flow"),
    72: ("Removals from Alberta", "DISPOSITION", None, "flow"),
    75: ("Reporting Adjustment", "DISPOSITION", None, "flow"),
    78: ("TOTAL OIL & EQUIVALENT DISPOSITION", "DISPOSITION", None, "flow"),
}

# --- Build the Data ---
all_records = []

for year in years:
    try:
        url = base_url.format(year=year)
        response = requests.get(url)
        wb = load_workbook(BytesIO(response.content), data_only=True)
        ws = wb.active

        for row_idx, row in enumerate(ws.iter_rows(min_row=1, max_row=100), start=1):
            if row_idx not in indent_map:
                continue

            label = row[1].value  # Column B
            if not label or not isinstance(label, str):
                continue

            top, cat1, cat2, typ = indent_map[row_idx]
            for i, cell in enumerate(row[3:15], start=1):  # Columns D to O
                val = cell.value
                if val is None:
                    continue
                all_records.append({
                    "Year": year,
                    "Month": i,
                    "Date": pd.to_datetime(f"{year}-{i:02d}-01"),
                    "Label": label.strip(),
                    "Top Level": top,
                    "Category 1": cat1,
                    "Category 2": cat2,
                    "Type": typ,
                    "Value": val
                })

        print(f"✅ Processed {year}")

    except Exception as e:
        print(f"❌ Failed to process {year}: {e}")

# --- Save and Push to GitHub ---
final_st3_df = pd.DataFrame(all_records)

push_df_to_github(
    df=final_st3_df,
    user="blainehodder",
    repo="WCSB_Supply_Demand",
    path="clean_data/st3/st3_cleaned.csv",
    commit_message="Upload cleaned ST3 data (multi-year)"
)

print("✅ ST3 cleaning and upload complete.")


Cloning into 'WCSB_Supply_Demand'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 148 (delta 37), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 4.56 MiB | 6.31 MiB/s, done.
Resolving deltas: 100% (37/37), done.
✅ Processed 2017
✅ Processed 2018
✅ Processed 2019
✅ Processed 2020
✅ Processed 2021
✅ Processed 2022
✅ Processed 2023
✅ Processed 2024
✅ File pushed to GitHub: https://github.com/blainehodder/WCSB_Supply_Demand/blob/main/clean_data/st3/st3_cleaned.csv
✅ ST3 cleaning and upload complete.
